In [42]:
import os
import pandas as pd
import json
import glob
import sklearn
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import  classification_report
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor

In [16]:
result=pd.read_csv("processed_data.csv")

In [17]:
result

,Unnamed: 0,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,...,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin,Dest,round_time
0,0,11,318,176,0.1,10,1023,100,47,17,...,17.0,10397,13204,101.0,51,0.0,10.0,ATL,MCO,0
1,1,18,315,122,0.0,10,1024,100,38,28,...,6.0,10397,13204,728.0,720,0.0,8.0,ATL,MCO,600
2,2,18,315,122,0.0,10,1024,100,38,28,...,37.0,10397,13930,753.0,725,1.0,28.0,ATL,ORD,600
3,3,18,317,122,0.0,10,1024,100,38,26,...,0.0,10397,13303,838.0,837,0.0,1.0,ATL,MIA,700
4,4,18,317,122,0.0,10,1024,100,38,26,...,0.0,10397,12892,922.0,915,0.0,7.0,ATL,LAX,700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851431,1851431,20,303,113,0.0,10,1014,0,58,22,...,9.0,14771,13930,513.0,519,0.0,0.0,SFO,ORD,2300
1851432,1851432,20,303,113,0.0,10,1014,0,58,22,...,5.0,14771,10397,615.0,632,0.0,0.0,SFO,ATL,2300
1851433,1851433,20,303,113,0.0,10,1014,0,58,22,...,0.0,14771,12478,735.0,750,0.0,0.0,SFO,JFK,2300
1851434,1851434,20,303,113,0.0,10,1014,0,58,22,...,0.0,14771,12478,655.0,704,0.0,0.0,SFO,JFK,2300


In [19]:
result['ArrDel15']

0          0.0
1          0.0
2          1.0
3          0.0
4          0.0
          ... 
1851431    0.0
1851432    0.0
1851433    0.0
1851434    0.0
1851435    0.0
Name: ArrDel15, Length: 1851436, dtype: float64

In [20]:
result=result[result['ArrDel15']==1.0]

In [21]:
result

,Unnamed: 0,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,...,DepDelayMinutes,OriginAirportID,DestAirportID,ArrTime,CRSArrTime,ArrDel15,ArrDelayMinutes,Origin,Dest,round_time
2,2,18,315,122,0.0,10,1024,100,38,28,...,37.0,10397,13930,753.0,725,1.0,28.0,ATL,ORD,600
10,10,18,317,122,0.0,10,1024,100,38,26,...,43.0,10397,11292,914.0,858,1.0,16.0,ATL,DEN,700
41,41,17,317,122,0.0,10,1025,100,38,20,...,67.0,10397,12892,1349.0,1230,1.0,79.0,ATL,LAX,1100
58,58,17,317,122,0.0,10,1025,100,38,21,...,37.0,10397,11292,1440.0,1408,1.0,32.0,ATL,DEN,1300
59,59,17,317,122,0.0,10,1025,100,38,21,...,10.0,10397,12892,1537.0,1520,1.0,17.0,ATL,LAX,1300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851396,1851396,24,292,113,0.0,10,1015,11,56,26,...,131.0,14771,14747,2025.0,1819,1.0,126.0,SFO,SEA,1600
1851397,1851397,25,293,113,0.0,10,1014,5,57,27,...,14.0,14771,14747,1901.0,1845,1.0,16.0,SFO,SEA,1700
1851399,1851399,25,293,113,0.0,10,1014,5,57,27,...,262.0,14771,11292,34.0,2031,1.0,243.0,SFO,DEN,1700
1851403,1851403,25,293,113,0.0,10,1014,5,57,27,...,67.0,14771,12889,1938.0,1850,1.0,48.0,SFO,LAS,1700


In [22]:
target=result["ArrDelayMinutes"]

In [23]:
features=result.copy()
features=features.drop(["ArrDel15","Unnamed: 0","date","airport","air","FlightDate","Origin","Dest"],axis=1)

In [24]:
features=features.drop(["ArrDelayMinutes","time","ArrTime","CRSArrTime","time","round_time"],axis=1)

In [25]:
features

,windspeedKmph,winddirDegree,weatherCode,precipMM,visibility,pressure,cloudcover,DewPointF,WindGustKmph,tempF,...,Quarter,Year,Month,DayofMonth,DepTime,DepDel15,CRSDepTime,DepDelayMinutes,OriginAirportID,DestAirportID
2,18,315,122,0.0,10,1024,100,38,28,40,...,1,2016,1,1,652.0,1.0,615,37.0,10397,13930
10,18,317,122,0.0,10,1024,100,38,26,41,...,1,2016,1,1,803.0,1.0,720,43.0,10397,11292
41,17,317,122,0.0,10,1025,100,38,20,45,...,1,2016,1,1,1137.0,1.0,1030,67.0,10397,12892
58,17,317,122,0.0,10,1025,100,38,21,47,...,1,2016,1,1,1322.0,1.0,1245,37.0,10397,11292
59,17,317,122,0.0,10,1025,100,38,21,47,...,1,2016,1,1,1325.0,0.0,1315,10.0,10397,12892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851396,24,292,113,0.0,10,1015,11,56,26,66,...,3,2017,9,30,1815.0,1.0,1604,131.0,14771,14747
1851397,25,293,113,0.0,10,1014,5,57,27,67,...,3,2017,9,30,1654.0,0.0,1640,14.0,14771,14747
1851399,25,293,113,0.0,10,1014,5,57,27,67,...,3,2017,9,30,2125.0,1.0,1703,262.0,14771,11292
1851403,25,293,113,0.0,10,1014,5,57,27,67,...,3,2017,9,30,1827.0,1.0,1720,67.0,14771,12889


In [26]:
train_feature,test_feature,train_target,test_target=train_test_split(features,target,test_size=0.2,random_state=42)

In [29]:
#linear regression
lin_reg=LinearRegression().fit(train_feature,train_target)
lin_reg_pred=lin_reg.predict(test_feature)

In [49]:
print("mean_squared_error:  ",mean_squared_error(test_target,lin_reg_pred))
print("mean_absolute_error: ",mean_absolute_error(test_target,lin_reg_pred))
print("r2_score:            ",r2_score(test_target,lin_reg_pred))

mean_squared_error:   315.470320932593
mean_absolute_error:  12.297523203305888
r2_score:             0.9369461237531438


In [37]:
#linear support vector regressor
SVR=LinearSVR().fit(train_feature,train_target)
SVR_pred=SVR.predict(test_feature)

In [50]:
print("mean_squared_error:  ",mean_squared_error(test_target,SVR_pred))
print("mean_absolute_error: ",mean_absolute_error(test_target,SVR_pred))
print("r2_score:            ",r2_score(test_target,SVR_pred))

mean_squared_error:   480.0506395212035
mean_absolute_error:  15.466166509904577
r2_score:             0.9040510260137536


In [40]:
#Extra Tree Regressor
extra_tree=ExtraTreesRegressor(n_estimators=100, random_state=0).fit(train_feature,train_target)
extra_tree_pred=extra_tree.predict(test_feature)

In [51]:
print("mean_squared_error:  ",mean_squared_error(test_target,extra_tree_pred))
print("mean_absolute_error: ",mean_absolute_error(test_target,extra_tree_pred))
print("r2_score:            ",r2_score(test_target,extra_tree_pred))

mean_squared_error:   292.384034581959
mean_absolute_error:  11.975189596969539
r2_score:             0.9415604400484108


In [43]:
#Gradient Boosting Regressor
XGBoost=GradientBoostingRegressor().fit(train_feature,train_target)
XGBoost_pred=XGBoost.predict(test_feature)

In [52]:
print("mean_squared_error:  ",mean_squared_error(test_target,XGBoost_pred))
print("mean_absolute_error: ",mean_absolute_error(test_target,XGBoost_pred))
print("r2_score:            ",r2_score(test_target,XGBoost_pred))

mean_squared_error:   291.77225130715715
mean_absolute_error:  11.776415587799706
r2_score:             0.9416827187679595
